In [44]:
import sys
import os
sys.path.append(os.path.abspath('./../../'))
sys.path.append(os.path.abspath('./../../../..'))
from Controllers.UrlController import UrlController
import json
from lib.UrlGenerator import UrlGenerator

In [2]:
controller = UrlController()
model = controller.get_urlModel(688)

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import os,sys
sys.path.append(os.path.abspath('./../../'))
from cell_extractor.diagnostics.ToolKit import get_DataFrame_from_query_result,find_equivalence,find_agreement,get_DataFrame_from_detection_df
from Controllers.MarkedCellController import MarkedCellController
from Controllers.SqlController import SqlController
import pandas as pd
from cell_extractor.CellAnnotationUtilities import CellAnnotationUtilities
from cell_extractor.AnnotationProximityTool import AnnotationProximityTool
from matplotlib.patches import Patch
from collections import Counter
from lib.UrlGenerator import UrlGenerator
from cell_extractor.QcGenerater import QcGenerater
from cell_extractor import utils 

animal = 'DK41'
controller = SqlController()
factor = controller.get_resolution(animal)

controller = MarkedCellController()
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':26,'FK_annotator_id':38}
mixed = get_DataFrame_from_query_result(controller.get_marked_cells(search_dict),'mixed',factor)

In [32]:
animal = 'DK41'
sures,unsure,nodetection = utils.get_single_threshold_sure_and_unsure(animal,2000,round=3)
detections = pd.concat([sures,unsure,nodetection])

suresr2 = get_DataFrame_from_detection_df(sures,'sures_round2')
unsuresr2 = get_DataFrame_from_detection_df(unsure,'unsures_round2')
nodetectionr2 = get_DataFrame_from_detection_df(nodetection,'no_detection_round2')

In [33]:
tool = AnnotationProximityTool()
tool.pair_distance=1
tool.set_annotations_to_compare(pd.concat([mixed,suresr2,unsuresr2]))
tool.find_equivalent_points()
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 7181
after removing duplicates 6681


[(('sures_round2',), 4955),
 (('unsures_round2',), 1226),
 (('mixed_Marissa_POSITIVE', 'sures_round2'), 242),
 (('mixed_Marissa_POSITIVE', 'unsures_round2'), 216),
 (('mixed_Marissa_NEGATIVE', 'unsures_round2'), 34),
 (('mixed_Marissa_NEGATIVE', 'sures_round2'), 8)]

In [63]:
sure_positive = tool.find_annotation_in_category([['mixed_Marissa_POSITIVE', 'sures_round2']])
unsure_positive = tool.find_annotation_in_category([['mixed_Marissa_POSITIVE', 'unsures_round2']])
sure_negative = tool.find_annotation_in_category([['mixed_Marissa_NEGATIVE', 'unsures_round2']])
unsure_negative = tool.find_annotation_in_category([['mixed_Marissa_NEGATIVE', 'sures_round2']])
from cell_extractor.utils import numpy_to_json

sure_positive = numpy_to_json(sure_positive[['x','y','section']].sort_values('section').to_numpy(),category='sure_positive',color_hex = '#2d652b')
unsure_positive = numpy_to_json(unsure_positive[['x','y','section']].sort_values('section').to_numpy(),category='unsure_positive',color_hex = '#5eb924')
sure_negative = numpy_to_json(sure_negative[['x','y','section']].sort_values('section').to_numpy(),category='sure_negative',color_hex = '#203891')
unsure_negative = numpy_to_json(unsure_negative[['x','y','section']].sort_values('section').to_numpy(),category='unsure_negative',color_hex = '#40d8ca')

In [64]:
urlGen = UrlGenerator()
urlGen.add_stack_image(animal,channel=1)
urlGen.add_stack_image(animal,channel=2,color='red')
urlGen.add_stack_image(animal,channel=3,color='green')
urlGen.add_annotation_layer('Sure Positive',annotations = sure_positive,color_hex='#2d652b')
urlGen.add_annotation_layer('Unsure Positive',annotations = unsure_positive,color_hex='#5eb924')
urlGen.add_annotation_layer('Sure Negative',annotations = sure_negative,color_hex='#203891')
urlGen.add_annotation_layer('Unsure Negative',annotations = unsure_negative,color_hex='#40d8ca')
sc_id = urlGen.add_to_database(f'DK41 unmixed QC Marissa',34)
print(f'meeting url id {animal} : https://activebrainatlas.ucsd.edu/ng/?id={sc_id}')

meeting url id DK41 : https://activebrainatlas.ucsd.edu/ng/?id=745
